In [1]:
import os
import sys
import pandas as pd

project_root = os.path.abspath("..")
if project_root not in sys.path:
    sys.path.append(project_root)

from src.generator.rag_generator import RAGGenerator
from src.utils.formatting import (
    format_retrieved_books,
    build_system_prompt,
    build_user_prompt,
)
from src.retriever.retriever import Retriever
from src.pipeline.adapters import retrieve_books_for_llm
from src.pipeline.rag_pipeline import RAGPipeline

#DATA_PATH = os.path.join(project_root, "data", "clean_books.csv")
#df = pd.read_csv(DATA_PATH)
df = pd.read_csv("../data/clean_books.csv")

/Users/biancaleoveanu/Desktop/rag-book-recommender/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
retriever = Retriever(
    index_path=os.path.join(project_root, "models", "faiss_index.bin"),
    metadata_path=os.path.join(project_root, "models", "metadata.pkl"),
    model_name="all-MiniLM-L6-v2",
    k=6,
)

Loading embedding model: all-MiniLM-L6-v2 on cpu
Loaded FAISS index from /Users/biancaleoveanu/Desktop/rag-book-recommender/models/faiss_index.bin
Loaded metadata from /Users/biancaleoveanu/Desktop/rag-book-recommender/models/metadata.pkl (n=10710)


In [3]:
gen = RAGGenerator(model="llama3")

user_query = "I want a cozy fantasy book with romance and a strong female lead."

raw_results = retriever.retrieve(user_query, k=6)
books = retrieve_books_for_llm(raw_results)

print("=== RETRIEVED BOOKS ===")
print(format_retrieved_books(books))

system_prompt = build_system_prompt("detailed")
user_prompt = build_user_prompt(user_query, books)

answer = gen.generate(system_prompt, user_prompt)
print(answer)


=== RETRIEVED BOOKS ===
1. Beauty's Release (Sleeping Beauty #3) by A.N. Roquelaure/Anne Rice | genres: adult fiction,erotica, fiction, fantasy, romance, erotica,bdsm, adult, fantasy,fairy tales, adult fiction, sexuality, womens fiction,chick lit | rating: 3.71
2. Seduction by Design by Erin St. Claire/Sandra Brown | genres: romance, romance,contemporary romance, fiction, contemporary, category romance,harlequin, adult, womens fiction,chick lit, mystery, adult fiction, novels | rating: 3.56
3. No Dress Rehearsal by Marian Keyes | genres: womens fiction,chick lit, fiction, short stories, novella, romance, fantasy,paranormal, european literature,irish literature | rating: 3.29
4. Girls' Night In by Lauren Henderson/Chris Manby/Sarah Mlynowski/Jill A. Davis/Stella Duffy/Lisa Jewell/Marian Keyes/Sophie Kinsella/Adèle Lang/Carole Matthews/Anna Maxted/Megan McCafferty/Alisa Valdes/Isabel Wolff/Meg Cabot/Jessica Adams/Louise Bagshawe/Emily Barr/Jenny Colgan/Lynda Curnyn/Jennifer Weiner | genr

In [4]:
user_query = "I want a cozy fantasy book with romance and a strong female lead."

raw_results = retriever.retrieve(user_query, k=6)   

retrieved = retrieve_books_for_llm(raw_results)    

print("=== RETRIEVED CANDIDATES ===")
print(format_retrieved_books(retrieved))

system_prompt = build_system_prompt(style="detailed")
user_prompt = build_user_prompt(user_query, retrieved, max_suggestions=3)

print("\n=== MODEL ANSWER ===\n")
answer = gen.generate(system_prompt, user_prompt)
print(answer)

=== RETRIEVED CANDIDATES ===
1. Beauty's Release (Sleeping Beauty #3) by A.N. Roquelaure/Anne Rice | genres: adult fiction,erotica, fiction, fantasy, romance, erotica,bdsm, adult, fantasy,fairy tales, adult fiction, sexuality, womens fiction,chick lit | rating: 3.71
2. Seduction by Design by Erin St. Claire/Sandra Brown | genres: romance, romance,contemporary romance, fiction, contemporary, category romance,harlequin, adult, womens fiction,chick lit, mystery, adult fiction, novels | rating: 3.56
3. No Dress Rehearsal by Marian Keyes | genres: womens fiction,chick lit, fiction, short stories, novella, romance, fantasy,paranormal, european literature,irish literature | rating: 3.29
4. Girls' Night In by Lauren Henderson/Chris Manby/Sarah Mlynowski/Jill A. Davis/Stella Duffy/Lisa Jewell/Marian Keyes/Sophie Kinsella/Adèle Lang/Carole Matthews/Anna Maxted/Megan McCafferty/Alisa Valdes/Isabel Wolff/Meg Cabot/Jessica Adams/Louise Bagshawe/Emily Barr/Jenny Colgan/Lynda Curnyn/Jennifer Weiner |

In [5]:
pipeline = RAGPipeline(retriever=retriever, generator=gen, k=6)

result = pipeline.run(
    "I want a cozy fantasy book with romance and a strong female lead.",
    style="detailed"
)

print("=== FINAL ANSWER ===\n")
print(result["answer"])

Device set to use mps:0


=== FINAL ANSWER ===

Based on your query for a cozy fantasy book with romance and a strong female lead, I recommend the following books from the retrieved list:

* **Beauty's Release** by A.N. Roquelaure/Anne Rice — This book is the third installment in the Sleeping Beauty trilogy, which features a strong female lead and fantasy elements. While it may not be strictly cozy, it has a romantic tone and explores themes of sexuality and empowerment.

* **Sleeping Beauty Trilogy (Sleeping Beauty #1-3)** by A.N. Roquelaure/Anne Rice — As the entire trilogy, this book set is a great match for your query. It features a strong female lead, fantasy elements, and romance throughout its three volumes.

I couldn't find another book in the retrieved list that perfectly matches all the criteria, but if you're interested in exploring more options, I suggest looking into **No Dress Rehearsal** by Marian Keyes, which has elements of fantasy and romance. However, it may not be as strongly focused on a st

In [6]:
print("\n=== RETRIEVED BOOKS USED ===\n")
print(format_retrieved_books(result["retrieved_books"]))


=== RETRIEVED BOOKS USED ===

1. Beauty's Release (Sleeping Beauty #3) by A.N. Roquelaure/Anne Rice | genres: adult fiction,erotica, fiction, fantasy, romance, erotica,bdsm, adult, fantasy,fairy tales, adult fiction, sexuality, womens fiction,chick lit | rating: 3.71
2. Seduction by Design by Erin St. Claire/Sandra Brown | genres: romance, romance,contemporary romance, fiction, contemporary, category romance,harlequin, adult, womens fiction,chick lit, mystery, adult fiction, novels | rating: 3.56
3. No Dress Rehearsal by Marian Keyes | genres: womens fiction,chick lit, fiction, short stories, novella, romance, fantasy,paranormal, european literature,irish literature | rating: 3.29
4. Girls' Night In by Lauren Henderson/Chris Manby/Sarah Mlynowski/Jill A. Davis/Stella Duffy/Lisa Jewell/Marian Keyes/Sophie Kinsella/Adèle Lang/Carole Matthews/Anna Maxted/Megan McCafferty/Alisa Valdes/Isabel Wolff/Meg Cabot/Jessica Adams/Louise Bagshawe/Emily Barr/Jenny Colgan/Lynda Curnyn/Jennifer Weiner